In [10]:
import pandas as pd
import redis
from redis.commands.search.field import TextField, NumericField
from redis.commands.search.index_definition import IndexDefinition, IndexType


In [2]:
df = pd.read_csv("hf://datasets/dubinthebox/movies/movies.csv")

/opt/anaconda3/envs/lli/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
r = redis.Redis(host="localhost", port=6379, decode_responses=True)

In [11]:
try:
    r.ft("movies").create_index(
        fields=[
            TextField("search_text"),
            TextField("title"),
            TextField("year"),
            TextField("genre"),
            TextField("director"),
            TextField("stars"),
            NumericField("rating"),
        ],
        definition=IndexDefinition(prefix=["movie:"], index_type=IndexType.HASH)
    )
except Exception as e:
    print("Index creation skipped or failed:", e)

In [12]:
for i, row in df.iterrows():
    search_text = f"{row['title']} {row['year']} {row['genre']} {row['director']} {row['stars']}"
    key = f"movie:{i}"
    r.hset(key, mapping={
        "search_text": search_text,
        "title": row["title"],
        "year": row["year"],
        "genre": row["genre"],
        "director": row["director"],
        "stars": row["stars"],
        "rating": row["rating"]
    })